# Lab 3: Stabilization and swing-up

In [1]:
using ControlSystems, LinearAlgebra, FurutaPendulums, Plots

In [2]:
# If you want to run it locally you can run a simulated version
# furuta = SimulatedFurutaPendulum()
# furuta = FurutaPendulum()
using FurutaPendulums, DifferentialEquations, LinearAlgebra, Plots
furuta = FurutaPendulum()

FurutaPendulum()

## Stabilization using model based on measured parameters
To stabilize the pendulum let us begin considering the linearized models obtained using the parameters from the manufacturer presented in lab 2, and let us initially neglect the effect of friction.

The model for the pendulum linearized around the downward equilibrium is then

In [4]:
A_down = [
    0.0  1.0     0.0     0.0
    0.0  0.0   -10.9962  0.0
    0.0  0.0     0.0     1.0
    0.0  0.0  -238.766   0.0
]

B_down = [
    0.0
 1474.4258
    0.0
 1472.1435
]

4-element Vector{Float64}:
    0.0
 1474.4258
    0.0
 1472.1435

The input to the above model is motor voltage in Volts (same unit as control signal) and the outputs are the angles in radians (same unit as measurement signal). Note that this is the friction-free model. How can you see that?

A: Can you read this? If yes then you wil also be able to see the volt.

See if you can arrive at the same A and B matrices from the manufacturer parameter values reported in lab 2. (If you cannot, ask for help, but don't get stuck here, instead move on with the matrices above).

A:

The demo controller of the Furuta pendulum uses a linear state feedback $u=-Kx$, where $x=[\varphi\ \dot{\varphi}\ \theta\ \dot{\theta}]^\top$, and where the closed loop poles are the eigenvalues, $\lambda(A-BK)=[-380, -40, -15, 0]^\top$.

The code skeleton below runs a state feedback control loop. Compute a pole placement (in Julia using ControlSystems.jl or in Matlab) so that you end up with the closed-loop poles above for the system linearized around its downward position. Before doing this, verify that you can achieve arbitrary pole placements by investigating controllability of the linearized systems.

A:

Run an experiment where the pendulum is first distrubed so it is swinging back-and-forth, and then the controller is activated. Compare the results you get from your pole placement designs with those you get for $K=0$.

To measure values we use the method `measure(furuta)` that is exported from the `FurutaPendulums` package. It returns a vector `[ϕ, dϕ, θ, dθ]` where the velocities are calculated similarly to what you have done so far, but with some added filtering to remove some noise. The arm angle is also corrected to have zero mean up, so for this case when we have the pendulum linearized around the downward position you will have to correct for this.

If you want to use Julia the [documentation](https://juliacontrol.github.io/ControlSystems.jl/latest/) for `ControlSystems.jl` is a good resource. Many things will be similar to Matlab, such as creating a state space using `ss` or placing closed loop poles using `place`.

In [5]:
# A 
# Ts = 0.00001
C = [1 1 1 1]
p=[-380, -40, -15, 0]
sys = ss(A_down, B_down, C, 0)

# State feedback controller
K = place(sys, p, :c)

1×4 Matrix{Float64}:
 0.0  0.678865  14.4424  -0.38443

In [7]:
# Example for control in downward equilibrium


# Disturb process
control(furuta, 1.0)
sleep(0.2)
control(furuta, 0.0)

h = 0.01
last_time = periodic_wait(furuta, 0.0, 0.0)
# Run controller

for i in 1:1500
    x = measure(furuta)
    x[3] = rem2pi(x[3] + pi, RoundNearest) # This will make sure that -pi < theta < pi, and 0 is down
    u = clamp(-dot(K, x), -5, 5) # We clamp the signal so nothing crazy is sent to the furuta
    control(furuta, u)
    last_time = periodic_wait(furuta, last_time, h)
end
control(furuta, 0.0)

-0.00015259021896696368

Now let us do the same thing for the upward equilibrium. For this to work we first need to swing the pendulum up. The code below does that for us, and then hands over to a linear state feedback controller. 

The swingup controller is an example of a nonlinear controller, as opposed to the linear state feedback controlllers we use to stabilize the pendulum. Its working principle is based on choosing the control signal to pump energy into the system. You can read more about how this is achieved [here](./papers/pendulum_swingup.pdf).

In [8]:
function swingup(x)
    C = 2000 
    K_arm = 0.00000387 # kinetic energy constant pendulum arm
    target_energy = 0.0047
    
    arm_energy = (K_arm * x[4]^2 + 0.0054 * 9.8 * 0.07/2 * (1 + cos(x[3])))
    dtheta = x[4] * (abs(x[4]) >= 2π / 2^11)
    return -C * (arm_energy - target_energy) * (dtheta * cos(x[3]) > 0 ? -1 : 1)
end

swingup (generic function with 1 method)

In [9]:
function create_feedback_controller(K)
    return x -> -dot(K, x) 
end

C = [1 1 1 1]
p=[-380, -40, -15, 0]
A_up = [
    0.0  1.0     0.0     0.0
    0.0  0.0   -10.9962  0.0
    0.0  0.0     0.0     1.0
    0.0  0.0  238.766   0.0
]

B_up = [
    0.0
 1474.4258
    0.0
 -1472.1435
]

sys = ss(A_up, B_up, C, 0)
K = place(sys, p, :c)

# This generates a function feedbackcontrol(x) = -dot(K, x) 
feedbackcontrol = create_feedback_controller(K) 

#1 (generic function with 1 method)

For the feedback controller to work you will need to replace $K=0$ with a meaningful controller. To do this, make a pole placement for the upward linearization that results in the same poles as the one you just tried out for downward control. The upwards linearization is very similar to the downwards one and only require negating the two parameters $A_{43}$ and $B_4$.

In [96]:
# A
# Disturb process
control(furuta, 1.0)
sleep(0.2)
control(furuta, 0.0)

h = 0.01
last_time = periodic_wait(furuta, 0.0, 0.0)
# Run controller

for i in 1:1500000
    x = measure(furuta)
    x = measure(furuta)
    x[3] = rem2pi(x[3], RoundNearest)
    x[1] = rem2pi(x[1], RoundNearest)
    if abs(rem2pi(x[3], RoundNearest)) < 0.3
        u = clamp(-dot(K, x), -4, 4) # We clamp the signal so nothing crazy is sent to the furuta
    else
        u = clamp(swingup(x), -4, 4) # We clamp the signal so nothing crazy is sent to the furuta
    end        
        control(furuta, u)
        last_time = periodic_wait(furuta, last_time, h)
end
control(furuta, 0.0)

LoadError: InterruptException:

Next write a simulation loop that starts running the `swingup` as long as we are not within some angle from upright position, and as soon as we are within this angle `-ν < θ < ν` it switches to run `feedbackcontrol`. A reasonable angle (in radians) can be around `ν=0.6`. Run it for long enough so you can see the pendulum being balanced for a while.

One tip is to also limit the control signal to some interval like $[-5, 5]$ to avoid huge signals.

In [50]:
# Disturb process
control(furuta, 1.0)
sleep(0.2)
control(furuta, 0.0)
h = 0.1
last_time = periodic_wait(furuta, 0.0, 0.0)
# Run controller

for i in 1:10000
    while abs(rem2pi(x[3], RoundNearest)) < 0.6
        x = measure(furuta)
        x = measure(furuta)
        u = clamp(-dot(K, x), -5, 5) # We clamp the signal so nothing crazy is sent to the furuta
        control(furuta, u)
#         last_time = periodic_wait(furuta, last_time, 0.001)
    end
    x = measure(furuta)
    x = measure(furuta)
    u = clamp(swingup(x), -4, 4) # We clamp the signal so nothing crazy is sent to the furuta
    control(furuta, u)
    last_time = periodic_wait(furuta, last_time, 0.01)
    x = measure(furuta)
end
control(furuta, 0.0)

LoadError: UndefVarError: swingup not defined

In [97]:
control(furuta, 0.0)

-0.00015259021896696368

## Control based on model identified from data
Now you should have controllers that work OK for the downward and upward equilibria. 

Take the identified linear models from Lab 2, the $A$ and $B$ matrices you identified there, and use the same design procedure. How do the resulting control performances compare? 

Also, investigate how your identified models differ from the ones used above by computing their poles and by plotting their Bode diagrams in the same figure. You can do this here in Julia using ControlSystems.jl, or if you prefer, in Matlab. 
To do it in Julia there is the `ControlSystems.jl` [documentation](https://juliacontrol.github.io/ControlSystems.jl/latest/), and a hint can be to look at the functions `ss`, `bodeplot` and `pzmap`. If you want to add to an existing figure instead of creating a new one you would instead use the function `bodeplot!`, similar to the Matlab `hold on`.

In [21]:
# A
A_up = [
    0.0  1.0     0.0     0.0
    0.0  0.0  -0.2403     0.0
    0.0  0.0     0.0     1.0
    0.0  0.0  -178.73   0.0
]

B_up = [
    0.0
 1474.4258
    0.0
 1472.1435
]

C = [1 1 1 1]
p=[-380, -40, -15, 0]
sys = ss(A_up, B_up, C, 0)
# State feedback controller
K = place(sys, p, :c)
function swingup(x)
    C = 2000 
    K_arm = 0.00000387 # kinetic energy constant pendulum arm
    target_energy = 0.0047
    
    arm_energy = (K_arm * x[4]^2 + 0.0054 * 9.8 * 0.07/2 * (1 + cos(x[3])))
    dtheta = x[4] * (abs(x[4]) >= 2π / 2^11)
    return -C * (arm_energy - target_energy) * (dtheta * cos(x[3]) > 0 ? -1 : 1)
end
function create_feedback_controller(K)
    return x -> -dot(K, x) 
end

C = [1 1 1 1]
p=[-380, -40, -15, -3]
A_up = [
    0.0  1.0     0.0     0.0
    0.0  0.0   -0.234  0.0
    0.0  0.0     0.0     1.0
    0.0  0.0   178   0.0
]

B_up = [
    0.0
 1474.4258
    0.0
 -1472.1435
]

sys = ss(A_up, B_up, C, 0)
K = place(sys, p, :c)

# This generates a function feedbackcontrol(x) = -dot(K, x) 
feedbackcontrol = create_feedback_controller(K)

#19 (generic function with 1 method)

In [ ]:
control(furuta, 1.0)
sleep(0.2)
control(furuta, 0.0)

h = 0.01
last_time = periodic_wait(furuta, 0.0, 0.0)
# Run controller

for i in 1:1500000
    x = measure(furuta)
    x = measure(furuta)
    x[3] = rem2pi(x[3], RoundNearest)
    x[1] = rem2pi(x[1], RoundNearest)
    if abs(rem2pi(x[3], RoundNearest)) < 0.3
        u = clamp(-dot(K, x), -5, 5) # We clamp the signal so nothing crazy is sent to the furuta
    else
        u = clamp(swingup(x), -5, 5) # We clamp the signal so nothing crazy is sent to the furuta
    end        
        control(furuta, u)
        last_time = periodic_wait(furuta, last_time, h)
end
control(furuta, 0.0)

Finally, pick the upward-stabilizing controller that turned out best so far, and now try to improve it manually by fine-tuning its parameters. You can do this by tuning the elements of $K$ directly (what are their physical interpretations?) or by moving the pole locations and re-doing the pole placement. In general, the faster the closed loop poles, the better control performance as long as the model matches reality well. If there is a mis-match, too fast poles will typically result in poorer performace. There is thus a trade-off decided by the quality of the model.

In [45]:
# A
K

1×4 Matrix{Float64}:
 0.0  -0.678865  -14.7667  -0.975405